# Robot drive and plot using jupyter
## Headers cell
This cell is for importing all needed  libraries and nodes will be used in the code 

In [5]:
%matplotlib widget

import ipywidgets as widgets 
import ipywidgets as GridBox 
import ipywidgets as Layout
from ipywidgets import ButtonStyle
from IPython.display import clear_output
from jupyros import ros3d
from IPython.display import display
import matplotlib.pyplot as plt
import jupyros as jr
import rospy

import os
import tf
from nav_msgs.msg import Odometry
from sensor_msgs.msg import LaserScan
from tf.transformations import quaternion_matrix
import numpy as np
from matplotlib.animation import FuncAnimation

from std_msgs.msg import String
from geometry_msgs.msg import Twist, Vector3
from assignment3.srv import robot_coordinates	
import actionlib
from move_base_msgs.msg import *
from actionlib_msgs.msg import *

## Set angular and linear velocities cell 
This cell is recieving the drive direction chosen by the user and take the action (angular and linear velocities)

In [6]:

def set_ang_lin_vel(direction_choice):
            #####  direction of movement  ##### 
    #                     up(i)     
    #        left(j)    stop(s)  right(l)
    #                     down(k)   
    

    
    #initialize the twist geometry_msg vector3 in x, y and z
    init = Vector3(0, 0, 0)
    total_vel = Twist(init,init)
    # Velocities initialization 
    v_lin = 0.5
    v_ang = 0.5
    # Action applied to the robot
    if direction_choice == "i":
        print("forwaard_pressed")
        total_vel.linear.x = v_lin
        total_vel.angular.z = 0
        
    elif direction_choice == "j":
        print("left pressed")
        total_vel.angular.z = v_ang
        total_vel.linear.x = 0
        
    elif direction_choice == "s":
        print("stop_pressed")
        total_vel.angular.z = 0
        total_vel.linear.x = 0
    elif direction_choice == "l":
        print("right_pressed")
        total_vel.angular.z = -v_ang
        total_vel.linear.x = 0

    elif direction_choice == "k":
        print("back_pressed")
        total_vel.linear.x = -v_lin
        total_vel.angular.z = 0
    
    
    #initialize the publisher
    pub = rospy.Publisher('/map_update_cmd_vel',Twist, queue_size=10)
    #public on topic map_update_cmd_vel to the robot
    pub.publish(total_vel)
    

## Drive interface cell 
This cell is responsible for displaying the buttons to let the user choose between modes and display the manual control keyboard

In [7]:
#call back functions for all on_button_click events
def drive_interface():
    #display all the buttons for manually drive the robot and change driving mode
    display(direction_box)
    display(obstacle_avoidance)
    display(button4)
    
    #display(buttonUP, buttonDOWN, buttonLEFT, buttonRIGHT, buttonPAUSE, obstacle_avoidance)   
    FORWARD_b.on_click(on_button_clicked_UP)
    BACKWARD_b.on_click(on_button_clicked_DOWN)
    STOP_b.on_click(on_button_clicked_PAUSE)
    GO_LEFT_b.on_click(on_button_clicked_LEFT)
    GO_RIGH_b.on_click(on_button_clicked_RIGHT)
   
    obstacle_avoidance.observe(obstacle_avoidance_handler, names='value')
    button4.on_click(on_button_clicked_back_to_menu)

    # display ros3d map
    display_map()
    

## Display ros3D map cell
This cell is responsible for displaying a realtime map of the robot its own working environment


In [8]:
def display_map():
    
    v = ros3d.Viewer()
    rc = ros3d.ROSConnection(url="ws://localhost:9090")
    tf_client = ros3d.TFClient(ros=rc, fixed_frame='map')

    laser_view = ros3d.LaserScan(topic="/scan", ros=rc, tf_client=tf_client)
    map_view = ros3d.OccupancyGrid(topic="/map", ros=rc, tf_client=tf_client, continuous=True)
    path = ros3d.Path(topic="/move_base/NavfnROS/plan", ros=rc,
    tf_client=tf_client)
    urdf = ros3d.URDFModel(ros=rc, tf_client=tf_client,
    path=os.environ.get('JUPYROS_ASSETS_URL', 'http://localhost:3000'))
    g = ros3d.GridModel()

    v.objects = [laser_view, map_view, urdf]
    display(v)

## Autonmous drive button cell
This cell is responsible for displaying input widges to let the user choose the x,y coordinates then the robot start moving towards it

In [9]:
def on_button_clicked_autonomous_drive(b):
    with output:
        button1.disabled=True
        button2.disabled=True
        #display the x and y input from user and start and back to main minu buttons
        display(x, y, button3, button4)
        button3.on_click(on_button_clicked_start_move)
        button4.on_click(on_button_clicked_back_to_menu)

## Display drive keyboard cell 
This cell is responsible for displaying the keyboard which the user will use to drive the robot manually 

In [10]:
def on_button_keyborad_display(b):
    with output:
        #Display keyboard interface to set directions of movement 
        button2.disabled=True
        button1.disabled=True
        
        drive_interface()

## Back to the main minu cell 
This cell is responsible for back to the main minu in case the user wants to change the mode of driving by disabling the whole buttons 

In [11]:
def on_button_clicked_back_to_menu(b):
    with output:
        clear_output()
        button1.disabled=False
        button2.disabled=False
        button3.disabled=False

## Start moving cell 
This cell is responsible for submitting the coordinats using services (start moving ) and wait for response for the robot to reach the coordinats 

In [12]:
#vector of the reached or not reached targets for the autonomous drive mode
reached_task = []                
def on_button_clicked_start_move(b):
    with output:
        print("Button 3")
        button3.disabled=True
        print("heading to: ",x.value, y.value)
        rospy.wait_for_service('robot_coordinates')
        Robot_coordinates = rospy.ServiceProxy('robot_coordinates', robot_coordinates)
        rt = Robot_coordinates(x.value , y.value)
        if rt.return_ == 1:
            print("target reached successfully!")
            reached_task.append('reached')
        else:
            print("time out, target not reached")
            reached_task.append('not reached')
        
        display(button4)
        button4.on_click(on_button_clicked_back_to_menu)
    

## Actions cell 
This cell is responsible for sending the right direction once the user pressed any direction button and change the parameter value once the user changed to drive manually with assistance 

In [13]:
def on_button_clicked_UP(b):
    set_ang_lin_vel("i")
def on_button_clicked_DOWN(b):
    set_ang_lin_vel("k")
def on_button_clicked_LEFT(b):
    set_ang_lin_vel("j")
def on_button_clicked_RIGHT(b):
    set_ang_lin_vel("l")
def on_button_clicked_PAUSE(b):
    set_ang_lin_vel("s")

#modify the value of the ROS parameter for collision avoidance control
def obstacle_avoidance_handler(b):
    rospy.set_param("/obstical_avoidance_with_keyboard", obstacle_avoidance.value)

## Node initialization cell

In [17]:
#intialization of the ros node 'user_interface'
rospy.init_node('user_interface')

## User GUI cell 
This cell is responsible for creating the whole buttons with special charactaristics and desplaying them

In [24]:
#creat all buttons used 
box_layout = widgets.Layout(display='flex',
                flex_flow='column',
                align_items='center',
                width='50%')


button1 = widgets.Button(description="case 1:autonomous")
button1.style.button_color = 'lightblue'
button2 = widgets.Button(description="case2: drive keyboard")
button2.style.button_color = 'lightblue'
button3 = widgets.Button(description="start the case")
button3.style.button_color = 'green'
button4 = widgets.Button(description="back to main menu")
button4.style.button_color = 'yellow'
#movement interface buttons
FORWARD_b = widgets.Button(description="FORWARD")
FORWARD_b.style.button_color = 'lightgreen'
BACKWARD_b = widgets.Button(description="BACKWARD")
BACKWARD_b.style.button_color = 'lightgreen'
STOP_b = widgets.Button(description="STOP")
STOP_b.style.button_color = 'red'
GO_LEFT_b = widgets.Button(description="LEFT")
GO_LEFT_b.style.button_color = 'lightgreen'
GO_RIGH_b = widgets.Button(description="right")
GO_RIGH_b.style.button_color = 'lightgreen'

# GridBox(children=[FORWARD_b, BACKWARD_b, STOP_b,GO_LEFT_b,GO_RIGH_b],
#      layout = Layout (
#         width='40%',
#         grid_template_rows='auto auto',
#         grid_template_columns='20% 20% 20% 20%',
#         grid_template_areas='''
#                     " .  FORWARD_b . "
#         " GO_LEFT_b . STOP_b STOP_b . GO_RIGH_b "
#                     " . BACKWARD_b ."
#          ''')
# )

direction_box = widgets.VBox(
    [widgets.HBox(children=[FORWARD_b],layout=box_layout),
    widgets.HBox( [ GO_LEFT_b  ,  STOP_b  ,   GO_RIGH_b ]),
    widgets.HBox( children=[BACKWARD_b],layout=box_layout)]
     )

obstacle_avoidance = widgets.Checkbox(
        value = False,
        description='active obstacles avoidance'
    )
    
#boxes for entering the coordinate of the taget to reach
x = widgets.FloatText(description="x coordinate: ")
y = widgets.FloatText(description="y coordinate: ")

#create an output cell
output = widgets.Output()

#display both choices buttons and output in the main minu
display(button1, button2, output)

#action done when pressing buttons 1,2
button1.on_click(on_button_clicked_autonomous_drive)
button2.on_click(on_button_keyborad_display)


Button(description='case 1:autonomous', style=ButtonStyle(button_color='lightblue'))

Button(description='case2: drive keyboard', style=ButtonStyle(button_color='lightblue'))

Output()

## Visualisers classes cell 
This cell is responsible for visualizing or displaying the plot of the robot position using the topic /pose and display the laserScan sensor output using polar plot.

In [9]:
class Visualiser:
    
    index = 0
    
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ln, = plt.plot([], [], 'ro')
        self.x_data, self.y_data = [] , []
    
    def plot_init(self):
        self.ax.set_xlim( -10, 10)
        self.ax.set_ylim( -10, 10)
        return self.ln
    
    def odom_callback(self, msg):
        
        self.index = self.index+1
        
        if self.index == 250:
            if len(self.y_data) < 500:
                self.y_data.append(msg.pose.pose.position.y)
            else:
                self.y_data.pop(0)
                self.y_data.append(msg.pose.pose.position.y)

            if len(self.x_data) < 500:
                self.x_data.append(msg.pose.pose.position.x)
            else:
                self.x_data.pop(0)
                self.x_data.append(msg.pose.pose.position.x)
                
            self.index = 0
        
            
    def update_plot(self, frame):
        #print("updateing")
        self.ln.set_data(self.x_data, self.y_data)
        return self.ln
   
#use of a polar plot for a simpler interpretation of the data
class Visualiser2:
    
    def __init__(self):
        self.fig, self.ax = plt.subplots( subplot_kw={'projection': 'polar'} )
        self.ln, = plt.plot([], [], 'r.' )
        self.theta_data = np.arange( 0, np.pi, np.pi/720.0)   
        self.d_data = []
    
    def plot_init(self):
        self.ax.set_rmax(15)
        return self.ln
    
        
    def laserScan_callback(self, msg):
            self.d_data = msg.ranges
    
    def update_plot(self, frame):
        #print("updateing")
        self.ln.set_data( self.theta_data, self.d_data )
        return self.ln

In [10]:
#visualization of the first plot --> position of the robot
vis = Visualiser()
sub = rospy.Subscriber('/odom', Odometry, vis.odom_callback)
ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init, blit=True)
plt.show(block=True)
plt.title("Robot position figure")

#visualization of the second plot --> data from LaserScan sensor
vis2 = Visualiser2()
sub2 = rospy.Subscriber('/scan', LaserScan, vis2.laserScan_callback)
ani2 = FuncAnimation(vis2.fig, vis2.update_plot, init_func=vis2.plot_init, blit=True)
plt.show(block=True)
plt.title("Laser scan figure")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.05, 'Laser scan figure')

## Histogram cell
This cell is used for plotting the reached tasks in autonumous mode

In [11]:
fig, ax = plt.subplots()
_, _, ln, = ax.hist([])

def init():
    return ln,

def update(frame):
    ax.hist(reached_task, bins=3, color='green')
    return ln,

ani3 = FuncAnimation(fig, update, init_func=init, blit=True)
plt.show()
plt.title("reached point")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'reached point')